In [11]:
# ==============================================================================
# CELL 1: SETUP, KONFIGURASI, DAN IMPOR PUSTAKA
# ==============================================================================

import xml.etree.ElementTree as ET
import random
import copy
import importlib.util
from collections import defaultdict, Counter
from deap import base, creator, tools
import time
import numpy as np

# Impor fungsi preferensi
try:
    import lecturer_preference_function_GITC
    import student_preference_function_GITC
    import institutional_preference_function_GITC
    importlib.reload(lecturer_preference_function_GITC)
    importlib.reload(student_preference_function_GITC)
    importlib.reload(institutional_preference_function_GITC)
    from lecturer_preference_function_GITC import count_preference_conflict as lecturer_conflict
    from student_preference_function_GITC import count_preference_conflict as student_conflict
    from institutional_preference_function_GITC import count_preference_conflict as institutional_conflict
except ImportError as e:
    print(f"Error importing preference functions: {e}")
    exit()

# Kategori dataset
DATASETS = {
    "Low": "agh-ggis-spr17_reduced.xml",
    "Middle": "muni-fsps-spr17c_reduced.xml",
    "High": "bet-spr18_reduced.xml"
}

# Konfigurasi Global
NUM_DAYS = 5
NUM_ROOMS = 44
NUM_SLOTS_PER_DAY = 8
TOTAL_SLOTS = NUM_DAYS * NUM_ROOMS * NUM_SLOTS_PER_DAY
MAX_SLOTS_PER_WEEK = NUM_DAYS * NUM_SLOTS_PER_DAY

# Variabel global untuk setiap kategori
lectures_to_be_scheduled = {}
lecture_schedule = {}
experiment_results = defaultdict(dict)

print("✅ Pustaka dan Konfigurasi Awal Siap.")

# ==============================================================================
# CELL 2: DEFINISI FUNGSI-FUNGSI HELPER DAN ANALISIS
# ==============================================================================

def parse_xml(file_path):
    """Mem-parsing file XML dan mengubahnya menjadi daftar data mata kuliah."""
    try:
        tree = ET.parse(file_path)
        root = tree.getroot()
    except (FileNotFoundError, ET.ParseError) as e:
        print(f"Error reading or parsing XML file {file_path}: {e}")
        return []
    lectures_data = []
    default_teacher_id_counter = 1000
    assigned_default_teachers = {}
    for course in root.findall('.//course'):
        course_id_str = course.get('id')
        if not course_id_str: continue
        try:
            course_id = int(course_id_str)
        except ValueError:
            continue
        course_teachers = [int(t.get('id')) for t in course.findall('./teacher') if t.get('id')]
        course_primary_teacher = 0
        if not course_teachers and course_id not in assigned_default_teachers:
            course_primary_teacher = default_teacher_id_counter
            assigned_default_teachers[course_id] = course_primary_teacher
            default_teacher_id_counter += 1
        elif course_id in assigned_default_teachers:
            course_primary_teacher = assigned_default_teachers[course_id]
        elif course_teachers:
            course_primary_teacher = course_teachers[0]
        for class_elem in course.findall('.//class'):
            class_id_str = class_elem.get('id')
            if not class_id_str: continue
            class_id = int(class_id_str)
            teacher1, teacher2 = 0, 0
            class_specific_teachers = [int(t.get('id')) for t in class_elem.findall('./teacher') if t.get('id')]
            if class_specific_teachers:
                teacher1 = class_specific_teachers[0]
                if len(class_specific_teachers) > 1: teacher2 = class_specific_teachers[1]
            elif course_teachers:
                teacher1 = course_teachers[0]
                if len(course_teachers) > 1: teacher2 = course_teachers[1]
            else:
                teacher1 = assigned_default_teachers.get(course_id, course_primary_teacher)
            lectures_data.append([class_id, course_id, teacher1, teacher2])
    return lectures_data

def create_lecture_schedule_map(lectures_list):
    """Membuat pemetaan dari ID kelas ke data mata kuliah."""
    return {lec[0]: (lec, 1) for lec in lectures_list}

def generate_random_schedule(lectures):
    """Menghasilkan satu individu jadwal acak."""
    data = [lec[0] for lec in lectures]
    if len(data) > TOTAL_SLOTS:
        raise ValueError(f"Penjadwalan tidak mungkin: Jumlah kuliah ({len(data)}) melebihi total slot ({TOTAL_SLOTS}).")
    empty_slots = [0] * (TOTAL_SLOTS - len(data))
    flat_schedule = data + empty_slots
    random.shuffle(flat_schedule)
    return flat_schedule

def to_2d_schedule(flat_schedule):
    """Mengubah jadwal 1D menjadi 2D (ruangan x slot waktu)."""
    return np.array(flat_schedule).reshape((NUM_ROOMS, NUM_DAYS * NUM_SLOTS_PER_DAY)).tolist()

def count_class_conflict(schedule_2d, lecture_schedule_map):
    """Menghitung konflik kelas (satu kelas di beberapa tempat sekaligus)."""
    total_conflicts = 0
    num_sessions = NUM_DAYS * NUM_SLOTS_PER_DAY
    for i in range(num_sessions):
        session_lectures = [schedule_2d[j][i] for j in range(NUM_ROOMS) if schedule_2d[j][i] != 0]
        if not session_lectures: continue
        class_ids = [lecture_schedule_map.get(l)[0][0] for l in session_lectures if lecture_schedule_map.get(l)]
        counts = Counter(class_ids)
        for count in counts.values():
            if count > 1: total_conflicts += (count - 1)
    return total_conflicts

def count_course_conflict(schedule_2d, lecture_schedule_map):
    """Menghitung konflik mata kuliah (kelas dari mata kuliah sama dijadwalkan sekaligus)."""
    total_conflicts = 0
    num_sessions = NUM_DAYS * NUM_SLOTS_PER_DAY
    for i in range(num_sessions):
        session_lectures = [schedule_2d[j][i] for j in range(NUM_ROOMS) if schedule_2d[j][i] != 0]
        if not session_lectures: continue
        course_ids = [lecture_schedule_map.get(l)[0][1] for l in session_lectures if lecture_schedule_map.get(l)]
        counts = Counter(course_ids)
        for count in counts.values():
            if count > 1: total_conflicts += (count - 1)
    return total_conflicts

def count_teacher_conflict(schedule_2d, lecture_schedule_map):
    """Menghitung konflik dosen (satu dosen mengajar di beberapa tempat sekaligus)."""
    total_conflicts = 0
    num_sessions = NUM_DAYS * NUM_SLOTS_PER_DAY
    for i in range(num_sessions):
        session_lectures = [schedule_2d[j][i] for j in range(NUM_ROOMS) if schedule_2d[j][i] != 0]
        if not session_lectures: continue
        teacher_lookup = defaultdict(int)
        for l_id in session_lectures:
            lec_data = lecture_schedule_map.get(l_id)
            if lec_data:
                for teacher in lec_data[0][2:]:
                    if teacher != 0: teacher_lookup[teacher] += 1
        for count in teacher_lookup.values():
            if count > 1: total_conflicts += (count - 1)
    return total_conflicts

def evaluate_feasibility(individual, category):
    """Fungsi evaluasi untuk Fase 1, menghitung total konflik hard."""
    schedule_2d = to_2d_schedule(individual)
    return (count_class_conflict(schedule_2d, lecture_schedule[category]),
            count_course_conflict(schedule_2d, lecture_schedule[category]),
            count_teacher_conflict(schedule_2d, lecture_schedule[category]))

def analyze_data_constraints(lectures, category):
    """Menganalisis batasan data input."""
    print(f"\n--- Menganalisis Batasan Data Input untuk Kategori {category} ---")
    teacher_counts = Counter()
    class_counts = Counter()
    impossible = False
    for lec in lectures:
        class_id, _, teacher1, teacher2 = lec
        class_counts[class_id] += 1
        if teacher1 != 0: teacher_counts[teacher1] += 1
        if teacher2 != 0: teacher_counts[teacher2] += 1
    for teacher, count in teacher_counts.items():
        if count > MAX_SLOTS_PER_WEEK:
            print(f"🔴 BATASAN MUSTAHIL (Dosen ID {teacher}): {count} jadwal, melebihi {MAX_SLOTS_PER_WEEK} slot.")
            impossible = True
    for class_id, count in class_counts.items():
        if count > MAX_SLOTS_PER_WEEK:
            print(f"🔴 BATASAN MUSTAHIL (Kelas ID {class_id}): {count} jadwal, melebihi {MAX_SLOTS_PER_WEEK} slot.")
            impossible = True
    if not impossible:
        print(f"✅ Tidak ditemukan batasan mustahil untuk kategori {category}.")
    return impossible

print("✅ Definisi Fungsi-Fungsi Helper Siap.")

# ==============================================================================
# CELL 3: DEFINISI FUNGSI-FUNGSI OPTIMISASI
# ==============================================================================

def find_feasible_schedule(lectures, category, pop_size=100, max_generations=200):
    """Menjalankan algoritma genetika Fase 1 untuk jadwal bebas konflik."""
    if hasattr(creator, "FitnessFeasible"): del creator.FitnessFeasible
    if hasattr(creator, "IndividualFeasible"): del creator.IndividualFeasible
    creator.create("FitnessFeasible", base.Fitness, weights=(-1.0, -1.0, -1.0))
    creator.create("IndividualFeasible", list, fitness=creator.FitnessFeasible)
    toolbox = base.Toolbox()
    toolbox.register("individual", tools.initIterate, creator.IndividualFeasible, lambda: generate_random_schedule(lectures))
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("evaluate", evaluate_feasibility, category=category)
    toolbox.register("mate", safe_crossover)
    toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.25)
    toolbox.register("select", tools.selTournament, tournsize=3)
    population = toolbox.population(n=pop_size)
    for ind in population:
        ind.fitness.values = toolbox.evaluate(ind)
    report_freq = max(1, max_generations // 10)
    for gen in range(max_generations):
        elite_count = 1
        elites = tools.selBest(population, k=elite_count)
        elites = [copy.deepcopy(ind) for ind in elites]
        offspring = toolbox.select(population, k=len(population))
        offspring = [copy.deepcopy(ind) for ind in offspring]
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < 0.8:
                toolbox.mate(child1, child2)
                del child1.fitness.values, child2.fitness.values
        for mutant in offspring:
            if random.random() < 0.2:
                toolbox.mutate(mutant)
                del mutant.fitness.values
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        for ind in invalid_ind:
            ind.fitness.values = toolbox.evaluate(ind)
        population[:] = offspring
        population[:elite_count] = elites
        if (gen + 1) % report_freq == 0:
            best_ind = tools.selBest(population, 1)[0]
            current_conflicts = sum(best_ind.fitness.values)
            print(f"Kategori {category} | Generasi {gen + 1}/{max_generations}... Konflik terendah: {current_conflicts}")
            if current_conflicts == 0:
                print(f"🎉 Solusi optimal (0 konflik) ditemukan untuk kategori {category}!")
                break
    best_feasible_ind = tools.selBest(population, 1)[0]
    return list(best_feasible_ind), best_feasible_ind.fitness.values

def run_preference_optimization(initial_schedule_flat, pref_function, category, pop_size=100, generations=100):
    """Menjalankan optimisasi Fase 2 dengan pelacakan generasi terbaik."""
    if hasattr(creator, "FitnessSingle"): del creator.FitnessSingle
    if hasattr(creator, "IndividualPreference"): del creator.IndividualPreference
    creator.create("FitnessSingle", base.Fitness, weights=(-1.0,))
    creator.create("IndividualPreference", list, fitness=creator.FitnessSingle)
    HARD_CONFLICT_PENALTY = 1000
    def evaluate_with_penalty(individual):
        schedule_2d = to_2d_schedule(individual)
        hard_conflicts_tuple = evaluate_feasibility(individual, category)
        total_hard_conflicts = sum(hard_conflicts_tuple)
        soft_conflicts = pref_function(schedule_2d, lecture_schedule[category])
        final_score = (total_hard_conflicts * HARD_CONFLICT_PENALTY) + soft_conflicts
        return (final_score,)
    toolbox = base.Toolbox()
    toolbox.register("individual", tools.initIterate, creator.IndividualPreference, lambda: generate_random_schedule(lectures_to_be_scheduled[category]))
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("evaluate", evaluate_with_penalty)
    toolbox.register("mate", safe_crossover)
    toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.2)
    toolbox.register("select", tools.selTournament, tournsize=5)
    population = [creator.IndividualPreference(copy.deepcopy(initial_schedule_flat)) for _ in range(int(0.2*pop_size))]
    population.extend(toolbox.population(n=pop_size - len(population)))
    hof = tools.HallOfFame(1)
    best_ever_score = float('inf')
    best_gen_found = 0
    for ind in population:
        ind.fitness.values = toolbox.evaluate(ind)
    report_freq = max(1, generations // 10)
    for gen in range(generations):
        offspring = toolbox.select(population, len(population))
        offspring = list(map(toolbox.clone, offspring))
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < 0.9:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values
        for mutant in offspring:
            if random.random() < 0.3:
                toolbox.mutate(mutant)
                del mutant.fitness.values
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        for ind in invalid_ind:
            ind.fitness.values = toolbox.evaluate(ind)
        population[:] = offspring
        hof.update(population)
        current_best_score = hof[0].fitness.values[0]
        if current_best_score < best_ever_score:
            best_ever_score = current_best_score
            best_gen_found = gen + 1
        if (gen + 1) % report_freq == 0:
            best_hard_conflicts = evaluate_feasibility(hof[0], category)
            best_soft_conflicts = best_ever_score % HARD_CONFLICT_PENALTY
            print(f"Kategori {category} | Generasi {gen + 1}/{generations} | Skor Terbaik: {best_ever_score:.1f} (Hard: {best_hard_conflicts}, Soft: {best_soft_conflicts:.1f})")
            if best_ever_score == 0:
                print(f"🎉 Solusi optimal (0 hard, 0 soft) ditemukan untuk kategori {category}!")
                break
    final_solution = hof[0]
    final_hard_conflicts = evaluate_feasibility(final_solution, category)
    final_score = final_solution.fitness.values[0]
    print(f"\n✅ Kategori {category} | Solusi terbaik dari Generasi {best_gen_found} dengan skor {final_score:.1f}")
    return final_solution, final_score, final_hard_conflicts

def safe_crossover(ind1, ind2):
    """Melakukan crossover tanpa menghilangkan mata kuliah."""
    lectures1 = [lec for lec in ind1 if lec != 0]
    lectures2 = [lec for lec in ind2 if lec != 0]
    if len(lectures1) != len(lectures2):
        return ind1, ind2
    unique_lectures = sorted(list(set(lectures1)))
    id_to_idx = {lec_id: i for i, lec_id in enumerate(unique_lectures)}
    idx_to_id = {i: lec_id for i, lec_id in enumerate(unique_lectures)}
    indices1 = [id_to_idx[lec_id] for lec_id in lectures1]
    indices2 = [id_to_idx[lec_id] for lec_id in lectures2]
    tools.cxOrdered(indices1, indices2)
    new_lectures1 = [idx_to_id[idx] for idx in indices1]
    new_lectures2 = [idx_to_id[idx] for idx in indices2]
    num_empty_slots = TOTAL_SLOTS - len(new_lectures1)
    empty_slots = [0] * num_empty_slots
    child1 = new_lectures1 + empty_slots
    child2 = new_lectures2 + empty_slots
    random.shuffle(child1)
    random.shuffle(child2)
    ind1[:] = child1
    ind2[:] = child2
    return ind1, ind2

print("✅ Definisi Fungsi-Fungsi Optimasi Siap.")

# ==============================================================================
# CELL 4: PEMROSESAN DATA DAN OPTIMISASI
# ==============================================================================

# Parameter penjadwalan
NUM_UNIQUE_TEACHERS = 40
NUM_LECTURES_TO_SCHEDULE = 75
initial_schedules = {}
results_by_category = {}

for category, xml_file in DATASETS.items():
    print(f"\n{'='*80}\n🚀 Memproses Kategori: {category}\n{'='*80}")
    
    # Memuat dan memilih data
    full_lectures = parse_xml(xml_file)
    all_unique_teacher_ids = sorted(list(set(lec[2] for lec in full_lectures if lec[2] != 0).union(set(lec[3] for lec in full_lectures if lec[3] != 0))))
    
    if len(all_unique_teacher_ids) >= NUM_UNIQUE_TEACHERS:
        target_teacher_ids = all_unique_teacher_ids[:NUM_UNIQUE_TEACHERS]
    else:
        print(f"PERINGATAN: Kategori {category} hanya memiliki {len(all_unique_teacher_ids)} dosen unik, kurang dari {NUM_UNIQUE_TEACHERS}. Menggunakan semua dosen.")
        target_teacher_ids = all_unique_teacher_ids
    
    target_teacher_ids_set = set(target_teacher_ids)
    print(f"Kategori {category}: Terpilih {len(target_teacher_ids_set)} dosen unik.")
    
    potential_lectures = [lec for lec in full_lectures if lec[2] in target_teacher_ids_set or lec[3] in target_teacher_ids_set]
    
    if len(potential_lectures) >= NUM_LECTURES_TO_SCHEDULE:
        lectures_to_be_scheduled[category] = potential_lectures[:NUM_LECTURES_TO_SCHEDULE]
    else:
        print(f"PERINGATAN: Kategori {category} hanya memiliki {len(potential_lectures)} mata kuliah, kurang dari {NUM_LECTURES_TO_SCHEDULE}. Menjadwalkan semua yang tersedia.")
        lectures_to_be_scheduled[category] = potential_lectures
    
    lecture_schedule[category] = create_lecture_schedule_map(lectures_to_be_scheduled[category])
    print(f"Kategori {category}: Total {len(lectures_to_be_scheduled[category])} kuliah akan dijadwalkan.")
    
    # Analisis batasan
    if analyze_data_constraints(lectures_to_be_scheduled[category], category):
        print(f"Program dihentikan untuk kategori {category} karena batasan mustahil.")
        continue
    
    # Fase 1: Mencari jadwal bebas konflik
    print(f"\n🚀 FASE 1: Mencari jadwal awal bebas konflik untuk kategori {category}...")
    pop_size_fase1 = 200
    generations_fase1 = 1000
    start_time_fase1 = time.time()
    initial_schedule, initial_hard_conflicts = find_feasible_schedule(
        lectures_to_be_scheduled[category],
        category,
        pop_size=pop_size_fase1,
        max_generations=generations_fase1
    )
    end_time_fase1 = time.time()
    initial_schedules[category] = initial_schedule
    print(f"\n✅ FASE 1 SELESAI untuk kategori {category} dalam {end_time_fase1 - start_time_fase1:.2f} detik.")
    print(f"Konflik Hard: {initial_hard_conflicts} (Class, Course, Teacher)")
    if sum(initial_hard_conflicts) > 0:
        print(f"🔴 PERINGATAN: Fase 1 untuk kategori {category} masih memiliki konflik.")
    
    # Fase 2: Optimisasi preferensi
    preference_functions = {
        "Lecturer": lecturer_conflict,
        "Student": student_conflict,
        "Institutional": institutional_conflict
    }
    results_by_category[category] = {}
    
    for pref_name, pref_func in preference_functions.items():
        print(f"\n🚀 FASE 2: Mengoptimalkan preferensi {pref_name} untuk kategori {category}...")
        pop_size_fase2 = 200
        generations_fase2 = 1000
        best_final_schedule, final_score, final_hard_conflicts = run_preference_optimization(
            initial_schedule,
            pref_func,
            category,
            pop_size=pop_size_fase2,
            generations=generations_fase2
        )
        final_soft_conflicts = pref_func(to_2d_schedule(list(best_final_schedule)), lecture_schedule[category])
        results_by_category[category][pref_name] = {
            "initial_hard_conflicts": initial_hard_conflicts,
            "final_schedule": best_final_schedule,
            "final_hard_conflicts": final_hard_conflicts,
            "final_soft_conflicts": final_soft_conflicts,
            "final_score": final_score
        }
        print(f"✅ FASE 2 SELESAI untuk preferensi {pref_name} di kategori {category}.")
        print(f"Konflik Hard Akhir: {final_hard_conflicts}, Konflik Soft ({pref_name}): {final_soft_conflicts}")

# ==============================================================================
# CELL 5: ANALISIS DAN TAMPILAN HASIL
# ==============================================================================

print("\n" + "="*80)
print("RINGKASAN HASIL KONFLIK PER KATEGORI")
print("="*80)

print(f"{'Kategori':<15} | {'Preferensi':<15} | {'Konflik Hard Awal':<20} | {'Konflik Hard Akhir':<20} | {'Konflik Soft Akhir':<20}")
print("-"*90)

for category in DATASETS.keys():
    if category not in results_by_category:
        print(f"Kategori {category}: Tidak ada hasil karena batasan mustahil.")
        continue
    for pref_name in results_by_category[category]:
        result = results_by_category[category][pref_name]
        initial_hard = sum(result["initial_hard_conflicts"])
        final_hard = sum(result["final_hard_conflicts"])
        final_soft = result["final_soft_conflicts"]
        print(f"{category:<15} | {pref_name:<15} | {initial_hard:<20} | {final_hard:<20} | {final_soft:<20}")

print("\n🏁 Analisis Selesai.")

✅ Pustaka dan Konfigurasi Awal Siap.
✅ Definisi Fungsi-Fungsi Helper Siap.
✅ Definisi Fungsi-Fungsi Optimasi Siap.

🚀 Memproses Kategori: Low
Kategori Low: Terpilih 40 dosen unik.
Kategori Low: Total 75 kuliah akan dijadwalkan.

--- Menganalisis Batasan Data Input untuk Kategori Low ---
✅ Tidak ditemukan batasan mustahil untuk kategori Low.

🚀 FASE 1: Mencari jadwal awal bebas konflik untuk kategori Low...
Kategori Low | Generasi 100/1000... Konflik terendah: 2.0
Kategori Low | Generasi 200/1000... Konflik terendah: 0.0
🎉 Solusi optimal (0 konflik) ditemukan untuk kategori Low!

✅ FASE 1 SELESAI untuk kategori Low dalam 284.44 detik.
Konflik Hard: (0.0, 0.0, 0.0) (Class, Course, Teacher)

🚀 FASE 2: Mengoptimalkan preferensi Lecturer untuk kategori Low...
Kategori Low | Generasi 100/1000 | Skor Terbaik: 0.0 (Hard: (0, 0, 0), Soft: 0.0)
🎉 Solusi optimal (0 hard, 0 soft) ditemukan untuk kategori Low!

✅ Kategori Low | Solusi terbaik dari Generasi 1 dengan skor 0.0
✅ FASE 2 SELESAI untuk p

Kategori High | Generasi 900/1000 | Skor Terbaik: 2000.0 (Hard: (0, 1, 1), Soft: 0.0)
Kategori High | Generasi 1000/1000 | Skor Terbaik: 2000.0 (Hard: (0, 1, 1), Soft: 0.0)

✅ Kategori High | Solusi terbaik dari Generasi 1 dengan skor 2000.0
✅ FASE 2 SELESAI untuk preferensi Institutional di kategori High.
Konflik Hard Akhir: (0, 1, 1), Konflik Soft (Institutional): 0

RINGKASAN HASIL KONFLIK PER KATEGORI
Kategori        | Preferensi      | Konflik Hard Awal    | Konflik Hard Akhir   | Konflik Soft Akhir  
------------------------------------------------------------------------------------------
Low             | Lecturer        | 0.0                  | 0                    | 0                   
Low             | Student         | 0.0                  | 0                    | 0                   
Low             | Institutional   | 0.0                  | 0                    | 0                   
Middle          | Lecturer        | 0.0                  | 0                    | 0     